# Алгоритм 

**Идея**: динамическое программирование по униграммной языковой модели + пост-обработка.  

1. **Словари частот**: загружаем RU/EN списки слов.  
   - logP(w) = log(freq/total).  
   - Спец-эвристики: бренды/модели, числа, пунктуация, дефисы/тире



3. **DP**: для каждой позиции *i* перебираем кандидатов j<i  
   выбираем разрез с максимальной суммой logP.  

4. **Пост-обработка**:  
   - Скобки `()[]{}`, «ёлочки», кавычки `"` и `'` (открывающие/закрывающие, апострофы внутри слов).  
   - Пунктуация `,.;:!?…` — убираем пробелы *перед*, ставим *после* при необходимости.  
   - Склеиваем `...` и юникодное `…`.  
   - Русские клитики: `-то` (без пробелов вокруг дефиса).  


In [1]:
import math
import requests
import os
import re
import string

## Глобальные настройки и словари
* Расширяем набор пунктуации (учитываем тире/многоточие/«ёлочки»).
* Определяем список «особых» терминов (бренды/модели и т.д.), чтобы не штрафовать их разбиение.
* Собираем special_mapping: конкатенированная форма → «нормальная». Это даёт большой приоритет распознаванию спец-слов.

In [2]:

punct = string.punctuation + '—–−…«»'
attaching_punct = set(punct) - {'-', '—', '–', '−'}

special_terms = [
    'USSR', 'ЕГЭ', 'iPhone', 'HUAWEI', 'Huawei', 'Playstation', 'PLAYSTATION',
    'Philips', 'HP', 'Samsung', 'Fender', 'Xbox One', 'Xiaomi', 'Indesit',
    'Merida', 'LG', 'Yamaha', 'Ikea', 'Asus', 'Dyson', 'Stels', 'Oppo', 'Honor',
    'Bosch', 'Adidas', 'TP-Link', 'JBL', 'Levi’s', 'Canon', 'Pixel', 'MSI',
    'Gibson', 'Atlant', 'TCL', 'Lenovo', 'Sharp', 'JVC', 'Acer', 'Dell', 'ROG',
    'Toshiba', 'Mac', 'OLED', 'LG OLED', 'A54', 'XR', 'RTX', 'Avito',
    'Яндекс Еда', 'Сбербанк', 'шкаф-купе', 'стиралок', 'вайфай', '5.1',
    'Samsung A54', 'вайфай 6'
]

special_mapping = {}
for t in special_terms:
    low_t = t.lower()
    concat = re.sub(r'[\s-]', '', low_t)
    special_mapping[concat] = t


### Загрузка словарей частот (ru/en)
* Скачиваем частотные списки слов (RU/EN) из открытого репозитория.
* Строим словари частот word_freq_ru и word_freq_en и сохраняем агрегаты (total_freq_*, max_freq_*).

In [ ]:

def get_word_freq(lang='ru'):
    if lang == 'ru':
        url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/ru/ru_full.txt"
    elif lang == 'en':
        url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2018/en/en_full.txt"
    else:
        raise ValueError("Unsupported language")
    
    response = requests.get(url)
    if response.status_code == 200:
        lines = response.text.splitlines()
        word_freq = {}
        for line in lines:
            parts = line.split(' ')
            if len(parts) == 2:
                word, freq = parts
                word_freq[word.lower()] = int(freq)
        return word_freq
    else:
        raise Exception(f"Failed to download {lang} frequency list")

print("Загружаю словари...")
word_freq_ru = get_word_freq('ru')
total_freq_ru = sum(word_freq_ru.values())
max_freq_ru = max(word_freq_ru.values()) if word_freq_ru else 1

word_freq_en = get_word_freq('en')
total_freq_en = sum(word_freq_en.values())
max_freq_en = max(word_freq_en.values()) if word_freq_en else 1


Загружаю словари...


## Считаем вероятность слова
* Если слово найдено в словаре — берём log(freq/total).
* Для спец-терминов — считаем вероятность очень высокой.
* Для цифр/пунктуации — отдельные бонусы (логика для 123, 5.1, дефисных форм).

In [ ]:

def get_unigram_logp(word, word_freq, total_freq, max_freq):
    alpha = 0.05
    epsilon = 1e-9
    if word in special_mapping:
        return math.log(max_freq * 1e10 / total_freq)
    if word in word_freq:
        return math.log(word_freq[word] / total_freq)
    elif word.isdigit():
        return math.log(max_freq * 10 / total_freq)
    elif len(word) > 0 and all(c in punct for c in word):
        return math.log(max_freq * 50 / total_freq)
    
    for sep in ['-', '–', '—']:
        if sep in word:
            parts = [p for p in word.split(sep) if p]
            if len(parts) > 1 and all(
                p and (p in word_freq or len(p) > 1 or p.isdigit())
                for p in parts
            ):
                sub_logps = [get_unigram_logp(p, word_freq, total_freq, max_freq)
                             for p in parts]
                sep_logp = math.log(max_freq * 50 / total_freq)
                return sum(sub_logps) + (len(parts) - 1) * sep_logp + math.log(1.01)
    
    if '.' in word:
        parts = [p for p in word.split('.') if p]
        if len(parts) > 1 and all(p.isdigit() for p in parts):
            sub_logps = [get_unigram_logp(p, word_freq, total_freq, max_freq)
                         for p in parts]
            sep_logp = math.log(max_freq * 50 / total_freq)
            return sum(sub_logps) + (len(parts) - 1) * sep_logp + math.log(1.01)
    
    return math.log(epsilon) + (len(word) - 1) * math.log(alpha)


## Пост-обработка индексов
Обрабатываем результат DP для корректной типографики.

**Скобки и кавычки:**
- Пробел перед открывающей, после неё убираем.
- Перед закрывающей пробел убираем, после ставим.  
  Пример: `" Вперёд "` -> `"Вперёд"`

**Знаки пунктуации:**
- Убираем пробелы перед `, . ; : ! ? …`.
- Ставим пробел после при необходимости.

**Дефис:**
- Убираем пробелы вокруг дефиса в конструкциях с `-то`.  
  Пример: `кто - то` -> `кто-то`

In [ ]:
OPEN_BRACKETS  = set('([{«')
CLOSE_BRACKETS = set(')]}»')
TRAILING_PUNCT = set(',.;:!?')
DOT = '.'
ELLIPSIS = '…'
SINGLE_QUOTE = "'"

def fix_bracket_and_quote_spacing(orig_text, indices):
    s = orig_text
    n = len(s)
    idx = {i for i in indices if 0 < i < n}

    for pos, ch in enumerate(s):
        if ch in OPEN_BRACKETS:
            idx.discard(pos + 1)
            if pos > 0 and (s[pos - 1].isalnum() or s[pos - 1] in CLOSE_BRACKETS or s[pos - 1] in {'"', "'"}):
                idx.add(pos)
        elif ch in CLOSE_BRACKETS:
            idx.discard(pos)
            if pos < n - 1 and (s[pos + 1].isalnum() or s[pos + 1] in OPEN_BRACKETS or s[pos + 1] in {'"', "'"}):
                idx.add(pos + 1)

    quote_open = True
    for pos, ch in enumerate(s):
        if ch == '"':
            if quote_open:
                idx.discard(pos + 1)
                if pos > 0 and (s[pos - 1].isalnum() or s[pos - 1] in CLOSE_BRACKETS):
                    idx.add(pos)
            else:
                idx.discard(pos)
                if pos < n - 1 and (s[pos + 1].isalnum() or s[pos + 1] in OPEN_BRACKETS):
                    idx.add(pos + 1)
            quote_open = not quote_open

    single_open = True
    for pos, ch in enumerate(s):
        if ch == "'":
            prev_ch = s[pos - 1] if pos > 0 else ''
            next_ch = s[pos + 1] if pos < n - 1 else ''
            if prev_ch.isalnum() and next_ch.isalnum():
                continue
            if single_open:
                idx.discard(pos + 1)
                if pos > 0 and (prev_ch.isalnum() or prev_ch in CLOSE_BRACKETS or prev_ch == '"'):
                    idx.add(pos)
            else:
                idx.discard(pos)
                if pos < n - 1 and (next_ch.isalnum() or next_ch in OPEN_BRACKETS or next_ch == '"'):
                    idx.add(pos + 1)
            single_open = not single_open

    return idx

def fix_punctuation_spacing(orig_text, idx):
    s = orig_text
    n = len(s)
    idx = set(i for i in idx if 0 < i < n)

    def is_opening(ch):
        return ch in OPEN_BRACKETS or ch in {'"', "'"}

    def is_closing(ch):
        return ch in CLOSE_BRACKETS or ch in {'"', "'"}

    for pos, ch in enumerate(s):
        if ch in TRAILING_PUNCT or ch in {DOT, ELLIPSIS}:
            idx.discard(pos)

    i = 0
    while i < n:
        if s[i] == DOT:
            j = i
            while j < n and s[j] == DOT:
                idx.discard(j)
                j += 1
            if j < n and (s[j].isalnum() or is_opening(s[j])):
                idx.add(j)
            i = j
        else:
            i += 1

    for pos, ch in enumerate(s):
        if ch == ELLIPSIS and pos < n - 1 and (s[pos + 1].isalnum() or is_opening(s[pos + 1])):
            idx.add(pos + 1)

    for pos, ch in enumerate(s):
        if ch in TRAILING_PUNCT and pos < n - 1 and (s[pos + 1].isalnum() or is_opening(s[pos + 1])):
            idx.add(pos + 1)

    for pos in range(n - 1):
        ch, nxt = s[pos], s[pos + 1]
        if (ch in TRAILING_PUNCT or ch in {DOT, ELLIPSIS} or is_opening(ch) or is_closing(ch)) and \
            (nxt in TRAILING_PUNCT or nxt in {DOT, ELLIPSIS} or is_opening(nxt) or is_closing(nxt)):
            idx.discard(pos + 1)

    return idx

def fix_ru_suffix_clitics(orig_text, idx):
    s = orig_text
    n = len(s)
    idx = set(i for i in idx if 0 < i < n)
    for i in range(1, n - 2):
        if s[i] == '-' and s[i+1:i+3].lower() == 'то' and s[i-1].isalpha():
            idx.discard(i)
            idx.discard(i + 1)
    return idx


# Динамическое программирование
- Цель: найти разбиение строки, которое максимизирует сумму лог-вероятностей.  
- Алгоритм перебирает разрезы до `max_word_len` символов назад и выбирает лучший.  
- Из массива `prev` восстанавливаются позиции пробелов.

In [ ]:
def restore_spaces_indices(text: str) -> list:
    orig_text = text
    text = text.lower()
    n = len(text)
    max_word_len = 25
    
    dp = [float("-inf")] * (n+1)
    prev = [-1] * (n+1)
    dp[0] = 0.0
    
    for i in range(1, n+1):
        for j in range(max(0, i - max_word_len), i):
            substr = text[j:i]
            if any(1040 <= ord(c) <= 1103 for c in substr):
                wf, tf, mf = word_freq_ru, total_freq_ru, max_freq_ru
            else:
                wf, tf, mf = word_freq_en, total_freq_en, max_freq_en
            
            logp = get_unigram_logp(substr, wf, tf, mf)
            score = dp[j] + logp
            if score > dp[i]:
                dp[i] = score
                prev[i] = j
    
    indices = []
    i = n
    while i > 0:
        j = prev[i]
        if j > 0:
            indices.append(j)
        i = j
    indices.reverse()

    idx = fix_bracket_and_quote_spacing(orig_text, indices)
    idx = fix_punctuation_spacing(orig_text, idx)
    idx = fix_ru_suffix_clitics(orig_text, idx)
    return sorted(idx)



# Обработка файлов
В строках .csv могут встречаться запятые, поэтому пишем свою реализацию чтения .csv

In [ ]:
def apply_indices(text: str, indices: list) -> str:
    out = []
    last = 0
    for i in indices:
        out.append(text[last:i])
        out.append(" ")
        last = i
    out.append(text[last:])
    return "".join(out)

def process_file(input_file: str, output_file: str):
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()
        output_data = []
        for line in lines[1:]:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',', 1)
            if len(parts) != 2:
                continue
            id_, text_no_spaces = parts
            indices = restore_spaces_indices(text_no_spaces)
            # restored_text = apply_indices(text_no_spaces, indices)
            output_data.append(f'{id_},"{str(indices)}"')
    
    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write("id,predicted_positions\n")
        for text in output_data:
            outfile.write(text + '\n')

In [ ]:
input_file = "dataset_1937770_3.txt"
output_file = "output.csv"

if not os.path.exists(input_file):
    raise FileNotFoundError(f"Файл {input_file} не найден")

process_file(input_file, output_file)
print(f"Файл сохранён как {output_file}")